## Written by Emanuel Calderon and Andy Leon

FYI: I know one group used a premium alphavantage api key, so you can just use that one from their code if they put it in

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import math
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 1. User Input and Data Collection

In [ ]:
# Function to search for ticker and obtain results: Process input and search results, then compare to verify existence of ticker
def tickerFunc(): 
    global ticker
    ticker = str(input("Enter a ticker (short character string): "))
    
    while True:
        if ticker.strip() == "":
            print("\nTicker should not be empty")
            tickerFunc()
        else:
            break
 
    ticker = ticker.upper()

    confirm = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords='+ticker+'&apikey='+key # call search endpoint utility, REPLACE WITH OWN KEY OR WHATEVER
    #confirm = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=tesco&apikey=demo' #DEMO
    confirmResp = requests.get(confirm)

    confirmData = confirmResp.json()

    try:
        confirmFrame = pd.DataFrame(confirmData["bestMatches"]) #constructing a DataFrame using the JSON
    except:
        print("\n## Daily limit for API key reached, or invalid API key! ##")
        return

    confirmFrame = confirmFrame

    if confirmFrame.empty == True:
        print("\nThere are no results for that ticker")
        tickerFunc()
    else:
        global tickercomp
        tickercomp = ticker

        for c in specialChars:
            tickercomp = tickercomp.replace(c,"")

        global tickerlist
        tickerlist = []
        for i in confirmFrame["1. symbol"]:
            for c in specialChars:
                i = i.replace(c,"") # writing clean search results to list for matching to ticker input, for any foreign market
            tickerlist.append(i)

In [ ]:
specialChars = ["!","@","^","=","/",",","."," "]

# Ask user to input API key: May use DN1PUQALCFSLB5MQ, replace wherever 'key' appears
key = str(input("Enter your API key for Alphavantage: "))

while True:
    if len(key.strip()) != 16:
        print("\nAPI key should be 16 characters long")
        key = str(input("Enter your API key for Alphavantage: "))
    else:
        break


print("\n### ANY NON-PREMIUM API KEY WILL ONLY HAVE 25 USES PER DAY ###\n")
tickerFunc()

# Verify inputted ticker exists in search results
try:
    while True:
        if tickercomp not in tickerlist:
            print("\nThis ticker does not appear in the database\n")
            tickerFunc()
        else:
            print("\nThis ticker appears in the database")
            break
except:
    print("## Replace API keys with your own and restart ##")

In [ ]:
# Request data from Alphavantage
asset = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+ticker+'&outputsize=full&apikey='+key #REPLACE WITH OWN KEY
spx = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=SPY&outputsize=full&apikey='+key #REPLACE WITH OWN KEY
# SPY needs to be used because alphavantage does not support indices

assetResp = requests.get(asset)
spxResp = requests.get(spx)
#assetResp, spxResp #to verify that we made successful calls

In [ ]:
assetData = assetResp.json()
#assetData #we can verify the data by outputting the JSON

spxData = spxResp.json()
#spxData #we can verify the data by outputting the JSON

In [ ]:
assetFrame = pd.DataFrame(assetData["Time Series (Daily)"]) #constructing a DataFrame using the JSON
assetFrame = assetFrame.T
assetDateRange = pd.DataFrame(assetFrame.loc['2017-12-31':'2017-01-01']) #constructing a new DataFrame with a specific date range 
assetDateRange.iloc[::-1].head()

In [ ]:
spxFrame = pd.DataFrame(spxData["Time Series (Daily)"]) #constructing a DataFrame using the JSON
spxFrame = spxFrame.T
spxDateRange = pd.DataFrame(spxFrame.loc['2017-12-31':'2017-01-01']) #constructing a new DataFrame with a specific date range 
spxDateRange.iloc[::-1].head()

In [ ]:
assetReturns = assetDateRange['4. close'].astype(float)#.pct_change() #astype is used here since all the data returned are strings

spxReturns = spxDateRange['4. close'].astype(float)#.pct_change()

In [ ]:
combined = pd.DataFrame([assetReturns,spxReturns],index=[ticker+' Close', 'SPY Close'])
combined = combined.T[::-1]
print(combined.head())
print("\nTotal Rows: ",len(combined.index))

In [ ]:
# Get rid of unused data
del asset
del assetResp
del assetData
del spx
del spxResp
del spxData

# 2. Percentage Changes in Daily Closing Price

In [ ]:
# Calculate and Display Daily Percentage Changes
combinedPC = combined.pct_change().dropna()
combinedPC = combinedPC.rename(columns = {ticker+" Close":ticker+" Percent Change (%)", "SPY Close":"SPY Percent Change (%)"})


for i in combinedPC.index:
    combinedPC.loc[i] = round(combinedPC.loc[i] * 100,3) # Display percent changes in percentage form
    
print(combinedPC.head())
print("\nTotal Rows: ",len(combinedPC.index))

In [ ]:
# Data may be incomplete, this code goes through the dates and returns only the index of the first day of a month with data
t = 0
combinedPCReset = combinedPC.reset_index()

datenums = []
datelist = []

datenums.append(combinedPCReset.index[0])
datelist.append(combinedPCReset["index"][0][5:7])

for i in combinedPCReset.index:    #Obtain first available day of the month, use the index of the row where it appears as the labels for x-axis bar graph
    t+=1
    if combinedPCReset["index"][t-1][5:7] in datelist[-1]:
        continue
    datenums.append(combinedPCReset.index[t-1])
    datelist.append(combinedPCReset["index"][t-1][5:7])

In [ ]:
# Create the bar graph
combinedPCbar = combinedPC.plot.bar(y = ticker+" Percent Change (%)", xlabel = "Date", ylabel = "Percentage", figsize=(12,4), rot = 45, color="orange")
combinedPCbar.set(xticks = datenums, title = ticker+" Daily Percentage Changes in 2017")
print(combinedPCbar)

In [ ]:
# Delete unused bar data
del combinedPCReset
del datelist
del combinedPCbar

In [ ]:
# Statistical calculations using daily percent change
assetValues = []
spxValues = []

assetValues.clear()
spxValues.clear()

#--------------------------------------------------
#--------------------------------------------------
# MEANS
    # Asset
meanSum = 0

for i in combinedPC[ticker+" Percent Change (%)"]:
    meanSum += i
    
assetMean = round(meanSum / len(combinedPC.index),3)
assetValues.append(assetMean)

    # SPY
meanSum = 0

for i in combinedPC["SPY Percent Change (%)"]:
    meanSum += i
    
spyMean = round(meanSum / len(combinedPC.index),3)
spxValues.append(spyMean)


#--------------------------------------------------
#--------------------------------------------------
# STANDARD DEVIATIONS
    # Asset
stdSum = 0

for i in combinedPC[ticker+" Percent Change (%)"]:
    stdSum = (i - assetMean) ** 2
    
assetStd = round(math.sqrt(stdSum / len(combinedPC.index)),3)
assetValues.append(assetStd)

    # SPY
stdSum = 0

for i in combinedPC["SPY Percent Change (%)"]:
    stdSum = (i - assetMean) ** 2
    
spxStd = round(math.sqrt(stdSum / len(combinedPC.index)),3)
spxValues.append(spxStd)


#--------------------------------------------------
#--------------------------------------------------
# VARIANCES
    # Asset
assetVar = round(assetStd ** 2,3)
assetValues.append(assetVar)

    # SPY
spxVar = round(spxStd ** 2,3)
spxValues.append(spxVar)


#--------------------------------------------------
#--------------------------------------------------
# MINS
    # Asset
min = combinedPC[ticker+" Percent Change (%)"][0]

for i in combinedPC[ticker+" Percent Change (%)"]:
    if i < min:
        min = i
        
assetMin = round(min,3)
assetValues.append(assetMin)

    # SPY
min = combinedPC["SPY Percent Change (%)"][0]

for i in combinedPC["SPY Percent Change (%)"]:
    if i < min:
        min = i
        
spxMin = round(min,3)
spxValues.append(spxMin)


#--------------------------------------------------
#--------------------------------------------------
# MAXS
    # Asset
max = combinedPC[ticker+" Percent Change (%)"][0]

for i in combinedPC[ticker+" Percent Change (%)"]:
    if i > max:
        max = i
        
assetMax = round(max,3)
assetValues.append(assetMax)

    # SPY
max = combinedPC["SPY Percent Change (%)"][0]

for i in combinedPC["SPY Percent Change (%)"]:
    if i > max:
        max = i
        
spxMax = round(max,3)
spxValues.append(spxMax)

combinedDict = {ticker+" Percent Change (%)":assetValues,"SPY Percent Change (%)":spxValues}

In [ ]:
# Display statistics
combinedStats = pd.DataFrame(combinedDict, index = ["Mean","Standard Deviation","Variance","Min","Max"])
combinedStats

# 3. Regression Analysis

In [ ]:
x = combinedPC.iloc[:, 0].values.reshape(-1,1)
y = combinedPC.iloc[:, 1].values.reshape(-1,1)
reg = linear_model.LinearRegression()
reg.fit(x, y)
prediction = reg.predict(x)

plt.xlabel(ticker)
plt.ylabel('S&P 500')
plt.scatter(combinedPC.iloc[:, 0], combinedPC.iloc[:, 1], color='red', marker='+')
plt.plot(combinedPC.iloc[:, 0], prediction, color='blue')

intercept = reg.intercept_[0]
coefficient = reg.coef_[0][0]
r2 = r2_score(y, prediction)
mse = mean_squared_error(y, prediction)

print(f'Intercept: {intercept}')
print(f'Coefficient: {coefficient}')
print(f'R2: {r2}')
print(f'Mean Squared Error: {mse}')

if coefficient > 0:
  print(f'{ticker} is positively correlated with the S&P 500.')
elif coefficient < 0:
  print(f'{ticker} is negatively correlated with the S&P 500.')
else:
  print(f'{ticker} has no correlation with the S&P 500.')